# Principal Component Analysis
pca is a mathematical operation that finds the most important features, or combination of features, contributing to your end value. In our case, we search for sensors that indicate, or indicate against, a future machine failure.

In [ ]:
import os
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split

In [ ]:
# check if the a directory exists, if not create it
outdir = './scratch'

if not os.path.exists(outdir):
    os.mkdir(outdir)

In [ ]:
# read our cleaned csv into a pandas dataframe
df = pd.read_csv(outdir +'/clean-df.csv')

#The index got reset, lets put the timestamp back
df['timestamp'] = pd.to_datetime(df['timestamp'])
df.set_index('timestamp', inplace=True)

### We can take a look at some of the sensors to better understand the real-time changes and break downs.

In [ ]:
df_subset = df[['sensor_03', 'sensor_04', 'sensor_21', 'machine_status']]

#Matplotlib allows us to define we want to use subplots for each columns (not all the lines on one plot) and the overall figure size.
df_subset.plot( kind='line',figsize=(12,6),subplots=True)
plt.tight_layout()
plt.show()

### Let's keep track of when those failures above happens. 

In [ ]:
failure_times = df[df['machine_status'] == 1]
print("Failure times:")
failure_times

### For our model, we'll need to designate which parts of the data should train the model and which should be used for testing. The validation split is a way of testing and checking against over-fitting, without using the actual test data.

### A common practice is to just split your data randomly, or by a defined percent. However, we want to make sure each split has at least one failure and recovery.

In [ ]:
# define the failures and specifically the first two
timestamps = list(failure_times.index)
first_failure = timestamps[0]
second_failure = timestamps[1]

# create a time buffer for our sensors to recover
two_hours = pd.Timedelta(seconds=60 * 120)

# split our large dataframe
# validation will include all the data up to the first failure (plus to hours)
# testing will include the following data up until the second failure (plus to hours)
# training data includes everything else
df_val = df.loc[:(first_failure + two_hours), :]
df_test = df.loc[(first_failure + two_hours):(second_failure + two_hours), :]
df_train = df.loc[(second_failure + two_hours):, :]

# get a better look at the results
print('Validation size:')
print(str(df_val.shape)+ '  ' + str(round(len(df_val) / len(df)*100,2)) + '%')
print('Test size:')
print(str(df_test.shape)+ '  ' + str(round(len(df_test) / len(df)*100,2)) + '%')
print('Train size:')
print(str(df_train.shape)+ '  ' + str(round(len(df_train) / len(df)*100,2)) + '%')


### For better training accuracy we should try and avoid big discrepancies between the sensor averages. Normalizing all the columns does the trick

In [ ]:
np.matrix(scaled_train).transpose

In [ ]:
# display original ranges
ranges = pd.DataFrame([{'min':i,'max':j} for i, j in zip(df_train.min(), df_train.max())])
col_names = df_train.max().keys()
ranges.index = col_names
display(ranges.transpose())

sensor_names = df_train.columns
min_max_scaler = MinMaxScaler(feature_range=(0, 1))

#finds the variables needed to transform our data to the [0,1] range we defined above
scaler = min_max_scaler.fit(df_train[sensor_names])

# transforms each dataframe
scaled_val = scaler.transform(df_val[sensor_names])
scaled_test = scaler.transform(df_test[sensor_names])
scaled_train = scaler.transform(df_train[sensor_names])

column_arrays = np.array(scaled_train).T
ranges = pd.DataFrame([{'min':k.min(),'max':k.max()} for k in column_arrays])
ranges.index = col_names
display(ranges.transpose())

### We're reading to run PCA

In [ ]:
pca = PCA(n_components=scaled_train.shape[1]).fit(scaled_train)

### Starting with a single feature, and each time a new feature is added, we will check to see how much variance within machine_status can be explained by the changes in the subset of features.

In [ ]:
plt.rcParams["figure.figsize"] = (12,6)

fig, ax = plt.subplots()
xi = np.arange(1, df_train.shape[1]-1, step=1)
y = np.cumsum(pca.explained_variance_ratio_)

plt.ylim(0.0,1.1)
plt.plot(xi, y, marker='o', linestyle='--', color='b')

plt.xlabel('Number of Components')
plt.xticks(np.arange(1, df_train.shape[1]+1, step=1)) #change from 0-based array index to 1-based human-readable label
plt.ylabel('Cumulative variance (%)')
plt.title('The number of components needed to explain variance')

plt.axhline(y=0.95, color='r', linestyle='-')
plt.text(25, 0.85, '95% cut-off threshold', color = 'red', fontsize=16)

ax.grid(axis='x')
plt.show()

### 95% accuracy is good enough for us

### Now, lastly, let's get those actual values we are correlating to the variance. Each Principal component contributes a different amount to the end goal, so we will include the ones the contribute most.

In [ ]:
num_pc_to_include = 12

def transform_df_pca(pca, data, scaled_data):
    """Reduce the number of features in the training data by the parameter num_features_to_include.

    :param pca: The PCA for the training data
    :type: PCA
    :param df_data: data dataframe
    :type: Pandas DataFrame
    :param scaled_data: Array of scaled data
    :type: ndarray
    :param num_pc_to_include: Number of features to include.  Currently, this number is determined
    by the num of PC's that were chosen by the PCA to reach 95% ( sum of explained_variance_ratio)
    :type: int
    
    :return: DataFrame with data that has been scaled and whose dimensions have been reduced.  This DataFrame has
    the same index as the param df_data
    :type: Pandas DataFrame
    """
    #Transform our raw data with the fitted pca numbers
    data_transformed = pca.transform(scaled_data)  # ndarray
    df_transformed = pd.DataFrame(data_transformed)
    
    #Rename the columns
    pcs = ['pc' + str(i + 1) for i in range(pca.n_components_)]
    df_transformed.columns = pcs

    #Provide our new numbers the same timestamps and machine_status as the original dataset
    df_transformed.index = data.index
    df_transformed = df_transformed[pcs[:num_pc_to_include]]
    df_transformed['machine_status'] = data['machine_status'].values
    

    return df_transformed

#Transform each dataset
df_train_pca = transform_df_pca(pca, df_train, scaled_train)
df_test_pca = transform_df_pca(pca, df_test, scaled_test)
df_val_pca = transform_df_pca(pca, df_val, scaled_val)

#### We can now think of our rows as vectors currently contributing to directional changes in machine_status

In [ ]:
df_train_pca